In [676]:
import pandas as pd
import numpy as np
import joblib

In [677]:
train_trans = pd.read_csv('data/train_trans.csv')

In [251]:
train_base = pd.read_csv('data/train_base.csv')
test_a_base = pd.read_csv('data/test_a_base.csv')

train_op = pd.read_csv('data/train_op.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')

train_trans = pd.read_csv('data/train_trans.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

train_label = pd.read_csv('data/train_label.csv')

## base处理

In [1054]:
train_base_no_nan = train_base.drop(columns=['service3_level'],axis=1)
# 离散值填充众数
train_base_no_nan['sex'].fillna('category 0',inplace=True)
train_base_no_nan['balance_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance1_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance2_avg'].fillna('level 1',inplace=True)

In [1055]:
user = train_base_no_nan['user'].values
train_base_no_user = train_base_no_nan.drop(columns=['user'],axis=1)

In [1056]:
from sklearn.preprocessing import OneHotEncoder

category_columns_name = ['sex','provider','level','verified','regist_type','agreement1','agreement2','agreement3','agreement4']

ohe = OneHotEncoder()
ohe_encoder = ohe.fit(train_base_no_user[category_columns_name])
one_hot_value = ohe_encoder.transform(train_base_no_user[category_columns_name]).toarray()

one_hot_df = pd.DataFrame(one_hot_value)

In [1057]:
train_base_categoty = train_base_no_user.select_dtypes('O')

for c in list(train_base_categoty.columns[0:9])+['service3']:
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))
    
for c in list(train_base_categoty.columns[11:17])+list(train_base_categoty.columns[18:25]):
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
d:\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [1058]:
train_base_categoty.drop(columns=['province','city'],axis=1,inplace=True)
train_base_categoty.drop(columns=category_columns_name,axis=1,inplace=True)

d:\anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [1059]:
train_base_categoty = (train_base_categoty-train_base_categoty.min())/(train_base_categoty.max()-train_base_categoty.min())

In [1060]:
train_base_categoty.head()

,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount
0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
1,0.90,0.9,0.00,0.0,0.95,1.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
2,0.30,0.2,0.65,0.5,0.00,1.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
3,0.50,0.3,0.00,0.0,0.55,0.75,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
4,0.35,0.4,0.00,0.0,0.40,1.00,0.0,0.166667,0.35,0.0,0.0,0.0,0.0


In [1061]:
# joblib.dump(ohe_encoder,'one_encoder.pkl')

In [1062]:
number_value = train_base_no_user.select_dtypes('int64').values
df_value= pd.DataFrame(number_value)
df_value.columns = train_base_no_user.select_dtypes('int64').columns

In [1063]:
df_value['product7_success_cnt'] = df_value['product7_cnt']-df_value['product7_fail_cnt']
df_value['card_cnt'] = df_value['card_a_cnt']+df_value['card_b_cnt']+df_value['card_c_cnt']+df_value['card_d_cnt']
df_value['ip_cnt_avg'] = df_value['ip_cnt']/df_value['login_days_cnt']
df_value['login_cnt_period']=df_value['login_cnt_period1']+df_value['login_cnt_period2']
df_value['login_cnt_period1_avg']=df_value['login_cnt_period1']/df_value['login_days_cnt']
df_value['login_cnt_period2_avg']=df_value['login_cnt_period2']/df_value['login_days_cnt']
df_value['login_cnt_period_avg']=df_value['login_cnt_period']/df_value['login_days_cnt']
df_value['using_time_avg'] = df_value['using_time']/df_value['login_days_cnt']
df_value['product7_success_cnt_avg'] = df_value['product7_success_cnt']/df_value['login_days_cnt']
df_value['product7_cnt_avg']=df_value['product7_cnt']/df_value['login_days_cnt']
df_value['product7_fail_cnt_avg']=df_value['product7_fail_cnt']/df_value['login_days_cnt']
df_value['service_cnt']=df_value['service1_cnt']+df_value['service2_cnt']
df_value['service_cnt_avg']= df_value['service_cnt']/df_value['login_days_cnt']
df_value['service_amt_cnt_avg']=df_value['service1_amt']/df_value['service_cnt']

In [1064]:
# 归一化
df_value = (df_value-df_value.min())/(df_value.max()-df_value.min())
# df_value = (df_value-df_value.mean())/df_value.std()

In [1065]:
df_value

,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg
0,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.000000,0.000000,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008
1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.048780,0.000000,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008
2,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.048780,0.000000,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008
3,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.048780,0.000000,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008
4,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.048780,0.000000,0.857143,0.120192,0.225519,0.002607,0.003001,0.000651,0.001898,0.811092,0.854204,0.773783,0.763667,0.0,0.823573,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47777,0.202759,0.245283,0.057692,0.098361,0.000000,0.028846,0.027397,0.0,0.252033,0.0,0.0,0.0,0.008970,0.011112,0.011357,0.074786,0.036239,0.178832,0.105691,0.051724,1.000000,0.057692,0.263237,0.011227,0.010506,0.010654,0.010575,0.773443,0.996049,0.763742,0.751947,0.0,0.800030,0.000008
47778,0.320000,0.037736,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.154472,0.0,0.0,0.0,0.001736,0.000565,0.000460,0.006944,0.012243,0.023723,0.048780,0.000000,0.857143,0.057692,0.239239,0.000515,0.000468,0.000346,0.000410,0.900208,0.856692,0.895139,0.884993,0.0,0.954417,0.000008
47779,0.244138,0.396226,0.057692,0.098361,0.000000,0.028846,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.001420,0.001604,0.026175,0.021058,0.045620,0.154472,0.112069,0.857143,0.057692,0.252993,0.001506,0.001241,0.001402,0.001316,0.912250,0.856276,0.896835,0.886694,0.0,0.932557,0.000008
47780,0.244138,0.245283,0.000000,0.000000,0.000000,0.120192,0.114155,0.0,0.300813,0.0,0.0,0.0,0.005498,0.020580,0.007203,0.045406,0.023996,0.167883,0.048780,0.000000,0.857143,0.000000,0.235239,0.014297,0.020220,0.006473,0.013765,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008


In [1066]:
# df_one_hot = pd.DataFrame(one_hot_value)

df_base = pd.DataFrame()
df_base['user'] = user

df_base = pd.concat([df_base, train_base_categoty,df_value,one_hot_df],axis=1)

In [1067]:
df_base.head()

,user,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,Train_06800,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,Train_23487,0.90,0.9,0.00,0.0,0.95,1.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,Train_36880,0.30,0.2,0.65,0.5,0.00,1.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,Train_35392,0.50,0.3,0.00,0.0,0.55,0.75,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,Train_35057,0.35,0.4,0.00,0.0,0.40,1.00,0.0,0.166667,0.35,0.0,0.0,0.0,0.0,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,0.857143,0.120192,0.225519,0.002607,0.003001,0.000651,0.001898,0.811092,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


## OP处理

In [86]:
train_op.head()

,user,op_type,op_mode,op_device,ip,net_type,channel,ip_3,tm_diff
0,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:22.000000000
1,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:21.000000000
2,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:23.000000000
3,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:26.000000000
4,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:41.000000000


In [656]:
op_df = pd.DataFrame()
group = train_op.groupby(['user']).count()
op_df['user'] = group.index

In [658]:
count_column=train_op.columns[1:-2]
for column in count_column:
    print(column)
    des_df = train_op.groupby('user')[column].describe()
    op_df['op_'+column+'_count'] = des_df['count'].values
    op_df['op_'+column+'_n'] = des_df['unique'].values
    op_df['op_'+column+'_fre'] = des_df['freq'].values

op_type
op_mode
op_device
ip
net_type
channel
ip_3


In [659]:
op_df_tmp = op_df

In [661]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

train_op['time_diff_sec'] = train_op['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    op_df['op_time_'+operate]=train_op.groupby('user')['time_diff_sec'].agg(operate).values

In [664]:
op_df.fillna(0,inplace=True)

In [665]:
op_df.head()

,user,op_op_type_count,op_op_type_n,op_op_type_fre,op_op_mode_count,op_op_mode_n,op_op_mode_fre,op_op_device_count,op_op_device_n,op_op_device_fre,op_ip_count,op_ip_n,op_ip_fre,op_net_type_count,op_net_type_n,op_net_type_fre,op_channel_count,op_channel_n,op_channel_fre,op_ip_3_count,op_ip_3_n,op_ip_3_fre,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std
0,Train_00000,102,11,44,102,12,44,91,2,89,90,3,82,30,3,28,102,4,70,90,3,82,683286.0,219874.0,3.058014e+05,263361.0,125051.535485
1,Train_00001,18,3,11,18,3,11,18,1,18,18,2,13,12,2,11,18,3,11,18,2,13,1263768.0,409809.0,6.598048e+05,438553.5,385614.886428
2,Train_00002,8,3,5,8,3,5,8,1,8,8,1,8,7,2,5,8,3,5,8,1,8,1283043.0,1282741.0,1.282845e+06,1282751.5,139.053690
3,Train_00003,108,8,42,108,8,41,93,2,89,93,18,30,50,2,46,108,3,51,93,5,31,1278388.0,199014.0,9.408690e+05,957412.0,247893.797669
4,Train_00004,5,2,4,5,2,4,5,1,5,5,1,5,4,1,4,5,2,4,5,1,5,631021.0,630973.0,6.309866e+05,630975.0,20.659138


## 交易处理

In [301]:
train_trans.head()

,user,platform,tunnel_in,tunnel_out,amount,type1,ip,type2,ip_3,tm_diff
0,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,185784,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:02:45.000000000
1,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,391769,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:03:58.000000000
2,Train_08351,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,27740,674e8d5860bc033d,f10a09fe9e522a47,11a213398ee0c623,ee386d6f9fe45d0d,18 days 11:06:49.000000000
3,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,36689,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 09:52:51.000000000
4,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,30746,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 07:50:05.000000000


In [713]:
trans_df = pd.DataFrame()
group = train_trans.groupby(['user']).count()
trans_df['user'] = group.index

### 离散列，全部统计有几个类别

In [684]:
count_column = list(train_trans.columns[1:4])+list(train_trans.columns[5:9]) 
for column in count_column:
    print(column)
    des_df = train_trans.groupby('user')[column].describe()
    trans_df['trans_'+column+'_count'] = des_df['count'].values
    trans_df['trans_'+column+'_n'] = des_df['unique'].values
    trans_df['trans_'+column+'_fre'] = des_df['freq'].values

platform
tunnel_in
tunnel_out
type1
ip
type2
ip_3


In [685]:
trans_df_tmp = trans_df

In [714]:
trans_df = trans_df_tmp

### 处理amount

In [715]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=train_trans.groupby(['user'])['amount'].agg(operate).values

### 处理时间

In [716]:
train_trans['time_diff_sec'] = train_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=train_trans.groupby('user')['time_diff_sec'].agg(operate).values

### 标准差、freq有空值，用0填充

In [717]:
trans_df.fillna(0,inplace=True)

In [689]:
trans_df.head()

,user,trans_type1,trans_type2,trans_platform_count,trans_platform_n,trans_platform_fre,trans_tunnel_in_count,trans_tunnel_in_n,trans_tunnel_in_fre,trans_tunnel_out_count,trans_tunnel_out_n,trans_tunnel_out_fre,trans_type1_count,trans_type1_n,trans_type1_fre,trans_ip_count,trans_ip_n,trans_ip_fre,trans_type2_count,trans_type2_n,trans_type2_fre,trans_ip_3_count,trans_ip_3_n,trans_ip_3_fre,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_00000,0,0,13,2,7,7,1,7,13,1,13,13,4,6,5,2,3,7,1,7,5,2,3,267542,24798,53330.307692,30746.0,65274.138488,2478067.0,1169773.0,1.553383e+06,1627246.0,3.970320e+05
1,Train_00001,0,0,2,2,1,1,1,1,2,1,2,2,2,1,1,1,1,1,1,1,1,1,1,36098,36098,36098.000000,36098.0,0.000000,1821027.0,289554.0,1.055290e+06,1055290.5,1.082915e+06
2,Train_00002,1,0,12,2,7,7,1,7,12,1,12,12,3,6,6,3,3,7,1,7,6,3,3,162423,24980,57329.583333,34541.5,49227.699521,2665430.0,575019.0,1.454428e+06,625491.0,1.069315e+06
3,Train_00003,2,1,11,2,9,9,1,9,11,1,11,11,5,4,8,2,7,9,1,9,8,2,7,188880,25315,61652.454545,36689.0,62929.509668,2563665.0,830115.0,2.243363e+06,2562619.0,6.976849e+05
4,Train_00004,2,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,36689,36689,36689.000000,36689.0,0.000000,2014022.0,2014022.0,2.014022e+06,2014022.0,0.000000e+00


## 数据合并

In [1068]:
train_df = pd.merge(df_base, train_label, on='user', how='left')
train_df = pd.merge(train_df, op_df, on='user', how='left')
train_df = pd.merge(train_df, trans_df, on='user', how='left')

In [1069]:
train = train_df

In [1070]:
length = len(train)
train_length = int(0.9*length)

x = train.drop(columns=['user','label'],axis=1)[0:train_length]
y = train['label'].values[0:train_length]

valid_x = train.drop(columns=['user','label'],axis=1)[train_length:]
valid_y = train['label'].values[train_length:]

In [1092]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

# lr = LogisticRegression()
# lr.fit(x,y)

model = LGBMClassifier(n_estimators=1000, learning_rate=0.04)
model.fit(x, y,
          eval_set=(valid_x, valid_y),
          early_stopping_rounds=3
          )

prediction = model.predict_proba(valid_x)[:,1]
auc = roc_auc_score(valid_y,prediction)
auc

[1]	valid_0's binary_logloss: 0.561894
Training until validation scores don't improve for 3 rounds
[2]	valid_0's binary_logloss: 0.559803
[3]	valid_0's binary_logloss: 0.557924
[4]	valid_0's binary_logloss: 0.556069
[5]	valid_0's binary_logloss: 0.554401
[6]	valid_0's binary_logloss: 0.55304
[7]	valid_0's binary_logloss: 0.551734
[8]	valid_0's binary_logloss: 0.550565
[9]	valid_0's binary_logloss: 0.549388
[10]	valid_0's binary_logloss: 0.548373
[11]	valid_0's binary_logloss: 0.547351
[12]	valid_0's binary_logloss: 0.546335
[13]	valid_0's binary_logloss: 0.545441
[14]	valid_0's binary_logloss: 0.544748
[15]	valid_0's binary_logloss: 0.543943
[16]	valid_0's binary_logloss: 0.543194
[17]	valid_0's binary_logloss: 0.54262
[18]	valid_0's binary_logloss: 0.54213
[19]	valid_0's binary_logloss: 0.541544
[20]	valid_0's binary_logloss: 0.540965
[21]	valid_0's binary_logloss: 0.540394
[22]	valid_0's binary_logloss: 0.53998
[23]	valid_0's binary_logloss: 0.539544
[24]	valid_0's binary_logloss: 0.

0.6791124149155936

In [1080]:
x.shape

(43003, 132)

In [567]:
joblib.dump(model,'model.pkl')

['model.pkl']

In [1082]:
importance_df = pd.DataFrame({
        'column': x.columns,
        'importance': model.feature_importances_,
    }).sort_values(by='importance')[-100:]

In [1084]:
select_feature=importance_df['column']

In [1086]:
x = x[select_feature]
valid_x = valid_x[select_feature]

In [1085]:
select_feature

100                 trans_type2
15                   card_a_cnt
120                trans_ip_3_n
23                 service1_amt
88             op_channel_count
53                            6
51                            4
49                            2
16                   card_b_cnt
71                           24
36             login_cnt_period
83                      op_ip_n
30               login_days_cnt
18                      op1_cnt
114                  trans_ip_n
99                  trans_type1
39         login_cnt_period_avg
55                            8
106         trans_tunnel_in_fre
5                  balance2_avg
9               product3_amount
92                    op_ip_3_n
4                      balance2
89                 op_channel_n
33         product7_success_cnt
82                  op_ip_count
58                           11
73             op_op_type_count
43        product7_fail_cnt_avg
84                    op_ip_fre
121              trans_ip_3_fre
67      

In [1083]:
importance_df

,column,importance
100,trans_type2,3
15,card_a_cnt,3
120,trans_ip_3_n,3
23,service1_amt,4
88,op_channel_count,4
53,6,5
51,4,5
49,2,6
16,card_b_cnt,6
71,24,7


In [1094]:
# from catboost import CatBoostClassifier


# model = CatBoostClassifier(
#     iterations=500,
#     random_seed=42,
#     logging_level='Silent'
# )

# model.fit(
#     x.values, y,
#     eval_set=(valid_x.values, valid_y),
# #     logging_level='Verbose',  # you can uncomment this for text output
#     plot=True
# )
# prediction = model.predict_proba(valid_x)[:,1]
# auc = roc_auc_score(valid_y,prediction)
# auc

baseline:
0.6769352059269422

添加trans amount的扩展变量：
0.6782946345739088

添加trans ip3的扩展变量：
0.6791192236191647

添加trans type直接数值化：
0.6796788342378327

添加所有的des特征，并修改了学习率：
0.6823851869318595

添加avg特征：
0.6831416556804824

In [597]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
